<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект. Анализ вакансий на HeadHunter

## (1) Работа с базой данных из Python

---

В реальной деятельности дата-сайентисту требуется не только писать запрос к данным, но и затем обрабатывать его результаты с помощью *Python*. Для решения таких задач требуется некоторое средство, которое будет связывать *Python* и *PostgreSQL* так, чтобы мы могли с помощью *Python* отправлять запросы в *Postgres* и принимать оттуда результаты.

Именно таким средством является пакет [psycopg2](https://www.psycopg.org/docs/), о котором мы сейчас и поговорим.

### Установка psycopg2

```linux
pip install psycopg2
```

Для подключения нам потребуются следующие данные:

* dbname — название базы, к которой нужно подключиться;
* user — имя пользователя в СУБД;
* password — пароль;
* host — адрес, по которому нужно подключиться;
* port — порт, к которому нужно подключиться (по умолчанию равен 5432).

Для нашей базы эти параметры сохранены в текущей директории в файле `config.ini`, который добавлен в список исключений `.gitignore` 

In [1]:
from bs4 import BeautifulSoup
import configparser
import pandas as pd
import psycopg2
import requests
import warnings

In [2]:
# Исключение предупреждающих сообщений из выходных данных ячеек
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
# Получение сведений об авторизации. Установление содинения с БД 
config = configparser.ConfigParser()
config.read("config.ini")
connection = psycopg2.connect(
    dbname=config.get("postgresql", 'DBNAME'),
    user=config.get("postgresql", "USER"),
    host=config.get("postgresql", "HOST"),
    password=config.get("postgresql", "PASSWORD"),
    port=config.get("postgresql", "PORT"),
)

Запрос:

In [15]:
# n = 10
# query = f"""
#     select *
#     from vacancies
#     limit {n}
# """

Выполнение запроса

In [14]:
# df = pd.read_sql_query(query, connection)
# connection.close()
# df

 <a name="anchor1"></a>
## (2) Введение. Знакомство с данными
---

Представьте, что вы устроились на работу в кадровое агентство, которое подбирает вакансии для IT-специалистов. Ваш первый проект — создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist. Сначала вам необходимо понять, что из себя представляют данные и насколько они соответствуют целям проекта. В литературе эта часть работы над ML-проектом называется Data Understanding, или анализ данных.

Наш проект включает в себя несколько этапов:

* знакомство с данными;
* предварительный анализ данных;
* детальный анализ вакансий;
* анализ работодателей;
* предметный анализ.

Каждая из частей будет состоять из блока практических заданий, которые вам необходимо выполнить в своих Jupyter-ноутбуках, и контрольных вопросов на платформе, которые проверяются автоматически.

Также вам предстоит отправить свой код ментору для код-ревью. Вам будет предоставлен ноутбук-шаблон и требования, согласно которым вы должны оформить своё решение.

Требования к оформлению ноутбука-решения:

* Решение оформляется только в Jupyter Notebook.
* Решение оформляется в соответствии с [ноутбуком-шаблоном](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@Project_2_%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD.ipynb).
* Каждое задание выполняется в отдельной ячейке, выделенной под задание (в шаблоне они помечены как ваш код здесь). Не следует создавать много ячеек для решения задачи — это провоцирует неудобства при проверке.
* Текст SQL-запросов и код на Python должны быть читаемыми. Не забывайте про отступы в SQL-коде.
* Выводы по каждому этапу оформляются в формате Markdown в отдельной ячейке (в шаблоне они помечены как ваши выводы здесь).
* Выводы можно дополнительно проиллюстрировать с помощью графиков. Они оформляются в соответствии с теми правилами, которые мы приводили в модуле по визуализации данных.
* Не забудьте удалить ячейку с данными соединения перед фиксацией работы в GitHub.



### Знакомство с данными

Все необходимые таблицы находятся в схеме public базы данных project_sql (именно эту базу вам необходимо указать в параметре dbname при подключении).

То есть параметры подключения будут следующими: см. файл `config.ini`

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_1.png)

Познакомимся с каждой таблицей.

vacancies

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_2.png)


Зарплатная вилка — это верхняя и нижняя граница оплаты труда в рублях (зарплаты в других валютах уже переведены в рубли). Соискателям она показывает, в каком диапазоне компания готова платить сотруднику на этой должности.

areas

Таблица-справочник, которая хранит код региона и его название.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_3.png)

employers

Таблица-справочник со списком работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_4.png)

industries

Таблица-справочник вариантов сфер деятельности работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_5.png)

employers_industries

Дополнительная таблица, которая существует для организации связи между работодателями и сферами их деятельности.

Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_6.png)

## (3) Предварительный анализ данных
---

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
    select 
        count(*)
    from
        vacancies
'''
df = pd.read_sql_query(query_3_1, connection)

In [6]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} вакансий.")

В базе содержится 49197 вакансий.


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [32]:
# текст запроса
query_3_2 = f'''
    select 
        count(*)
    from
        employers
'''
df = pd.read_sql_query(query_3_2, connection)

In [33]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} работодателей.")

В базе содержится 23501 работодателей.


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [35]:
# текст запроса
query_3_3 = f'''
    select 
        count(*)
    from
        areas
'''
df = pd.read_sql_query(query_3_3, connection)

In [37]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} региона.")

В базе содержится 1362 региона.


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [38]:
# текст запроса
query_3_4 = f'''
    select 
        count(*)
    from
        industries
'''
df = pd.read_sql_query(query_3_4, connection)

In [40]:
# результат запроса
print(f"В базе содержится {df.iloc[0, 0]} сферы деятельности.")

В базе содержится 294 сферы деятельности.


## (4) Детальный анализ ваканский
---

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [32]:
# текст запроса
query_4_1 = f'''
    select
        a.name as area,
        count(v.id) as cnt
    from
        areas as a
        join vacancies as v on a.id = v.area_id
    group by
        a.id
    order by
        2 desc
    limit
        5
'''
df = pd.read_sql_query(query_4_1, connection)

In [33]:
print("Количество вакансий по регионам РФ (5-ка лидеров)")
display(df)

Количество вакансий по регионам РФ (5-ка лидеров)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [5]:
# текст запроса
query_4_2 = f'''
    select
        count(*)
    from
        vacancies as v
    where
        v.salary_from is not null
        or v.salary_to is not null
'''
df = pd.read_sql_query(query_4_2, connection)

In [6]:
print(f"Количество вакансий, где заполнено хотя бы одно из полей с зарплатой, составляет {df.iloc[0, 0]}.")

Количество вакансий, где заполнено хотя бы одно из полей с зарплатой, составляет 24073.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [72]:
# текст запроса
query_4_3_1 = f'''
    select
        round(avg(salary_from))
    from
        vacancies
'''
query_4_3_2 = f'''
    select
        round(avg(salary_to))
    from
        vacancies
'''

mean_salary_from = int(pd.read_sql_query(query_4_3_1, connection).iloc[0, 0])
mean_salary_to = int(pd.read_sql_query(query_4_3_2, connection).iloc[0, 0])

In [79]:
print("Среднее значение зарплаты:")
print(f"    \u2022 для нижней границы составляет {mean_salary_from:,} \u20BD;".replace(",", " "))
print(f"    \u2022 для верхней границы составляет {mean_salary_to:,} \u20BD.".replace(",", " "))

Среднее значение зарплаты:
    • для нижней границы составляет 71 065 ₽;
    • для верхней границы составляет 110 537 ₽.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [6]:
query_4_4 = f'''
    select
        schedule,
        employment,
        count(*)
    from
        vacancies
    group by
        schedule,
        employment
    order by
        3 DESC
'''
df = pd.read_sql_query(query_4_4, connection)

In [7]:
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [15]:
print(f"На втором месте по популярности находится <{df.iloc[1, 0]} \u2014 {df.iloc[1, 1]}>")

На втором месте по популярности находится <Удаленная работа — Полная занятость>


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''
    select
        experience,
        count(*)
    from
        vacancies
    group by
        experience
    order by
        2
'''
df = pd.read_sql_query(query_4_5, connection)

In [21]:
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


## (5) Анализ работодателей
---

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
query_5_1 = f'''
    select
        e.name,
        count(*)
    from
        employers as e
        join vacancies as v on e.id = v.employer_id
    group by
        e.id,
        e.name
    order by
        2 DESC
'''
df = pd.read_sql_query(query_5_1, connection)

In [26]:
print(f"На первом и пятом месте по количеству вакансий находятся {df['name'].iloc[0]} и {df['name'].iloc[4]}.")

На первом и пятом месте по количеству вакансий находятся Яндекс и Газпром нефть.


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [18]:
query_5_2 = f'''
    with areas_vacancies_cnt as (
        select
            a.id,
            a.name as name,
            count(v.id) as vacancies_cnt
        from 
            areas as a
            left join vacancies as v on a.id = v.area_id
        group by
            a.id,
            a.name
    )
    select
        av.name,
        av.vacancies_cnt,
        count(e.id) as employers_cnt
    from
        areas_vacancies_cnt as av
        left join employers as e on e.area = av.id
    group by
        av.id,
        av.name,
        av.vacancies_cnt
    having
        vacancies_cnt = 0
    order by
        3 desc
    limit
        1

'''
df = pd.read_sql_query(query_5_2, connection)

In [19]:
df

,name,vacancies_cnt,employers_cnt
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
query_5_3 = f'''
    select
        e.name as employer_name,
        count(distinct v.area_id) as areas_cnt
    from
        employers as e
        join vacancies as v on e.id = v.employer_id
    group by
        e.id,
        e.name
    order by
        2 DESC
'''
df = pd.read_sql_query(query_5_3, connection)

In [28]:
df

,employer_name,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [7]:
query_5_4 = f'''
    select 
        count(distinct e.id)
    from 
        employers e
        left join employers_industries ei on e.id = ei.employer_id
    where 
        ei.employer_id IS NULL
'''
df = pd.read_sql_query(query_5_4, connection)

In [8]:
print(f"Количество работодателей, у которых не указана сфера деятельности, составляет {df.iloc[0, 0]}.")

Количество работодателей, у которых не указана сфера деятельности, составляет 8419.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [71]:
query_5_5 = f'''
    select
        e.name, 
        count(distinct ei.industry_id)
    from 
        employers e
        left join employers_industries ei on e.id = ei.employer_id
    group by
        e.id
    having
        count(distinct ei.industry_id) = 4
    order by
        1
    limit
        3 
    
'''
df = pd.read_sql_query(query_5_5, connection)

In [75]:
print("Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний,\n" 
      f"у которых указано четыре сферы деятельности, является <{df.iloc[2, 0]}>")

Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний,
у которых указано четыре сферы деятельности, является <2ГИС>


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [15]:
source_industry = "Разработка программного обеспечения"
query_5_6 = f'''
    select
        count(distinct e.id) as количество_работодателей
    from
        employers e
        join employers_industries ei on e.id = ei.employer_id
        join industries i on ei.industry_id = i.id
    where
        i.name = '{source_industry}';
'''
df = pd.read_sql_query(query_5_6, connection)

In [17]:
print(f"Количество работодателей, занимающихся разработкой программного обеспечения, составляет {df.iloc[0,0]}.")

Количество работодателей, занимающихся разработкой программного обеспечения, составляет 3553.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
url = "https://ru.wikipedia.org/wiki/Города-миллионеры_России"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
# Ищем всё с тегом table на листе.
table = soup.find_all("table")
# Опытным путем выяснилось, что искомая таблица имеет индекс 1.
rows = table[1].find_all("tr")[1:]
cities = [row.find_all("td")[1].text.strip() for row in rows]

In [45]:
# Плейсхолдеры для экранирования значений списка городов-миллионников (cities) в SQL-запросе.
placeholders = ', '.join(['%s'] * len(cities))
query_5_7 = f'''
    select
        name,
        cnt
    -- Подзапросы используются для сортировки (чтобы строка итогов была внизу),
    -- а остальная таблица была отсортирована по возрастанию количества вакансий
    -- как в примере в условии задания на платформе.
    from (  
        select
            a.name,
            count(v.id) as cnt,
            1 as order_col -- технический столбец для сортировки
        from
            areas as a
            join vacancies as v on a.id = v.area_id
            join employers as e on e.id = v.employer_id
        where
            a.name in ({placeholders}) and
            e.name = 'Яндекс'
        group by
            a.id,
            a.name
        union all -- формирование строки итогов
        select
            'Total' as name,
            count(v.id) as cnt,
            2 as order_col -- технический столбец для сортировки
        from
            areas as a
            join vacancies as v on a.id = v.area_id
            join employers as e on e.id = v.employer_id
        where
            a.name in ({placeholders}) and
            e.name = 'Яндекс'
    ) sub -- сортировка таблицы целиком
    order by
        order_col,
        cnt;
'''
df = pd.read_sql_query(query_5_7, connection, params=cities*2)

In [46]:
df

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


## (6) Предметный анализ

---

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [9]:
query_6_1 = f'''
    select
        count(distinct v.id) as cnt
    from
        vacancies as v
    where
        lower(v.name) like '%data%'
        or lower(v.name) like '%данн%' 
'''
df = pd.read_sql_query(query_6_1, connection)

In [10]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия, имеющая отношение к данным.")

В базе содержится 1771 вакансия, имеющая отношение к данным.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [27]:
query_6_2 = f'''
    select
        count(distinct v.id) as cnt
    from
        vacancies as v
    where
        (
            lower(v.name) like '%data scientist%'
            or lower(v.name) like '%data science%'
            or lower(v.name) like '%исследователь данных%'
            or lower(v.name) like '%machine learning%'
            or lower(v.name) like '%машинн%обучен%'
            or (v.name like '%ML%' and v.name not like '%HTML%') 
        ) 
        and (
            lower(v.name) like '%junior%' 
            or lower(v.experience) like '%нет опыта%'
            or lower(v.employment) like '%стажировка%' 
        )
'''
df = pd.read_sql_query(query_6_2, connection)

In [29]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для начинающего дата-сайентиста.")

В базе содержится 51 вакансия для начинающего дата-сайентиста.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [4]:
query_6_3 = f'''
    select
        count(distinct v.id) as cnt
    from
        vacancies as v
    where
        (
            lower(v.name) like '%data scientist%'
            or lower(v.name) like '%data science%'
            or lower(v.name) like '%исследователь данных%'
            or lower(v.name) like '%machine learning%'
            or lower(v.name) like '%машинн%обучен%'
            or (v.name like '%ML%' and v.name not like '%HTML%') 
        ) 
        and (
            lower(v.key_skills) like '%sql%' 
            or lower(v.key_skills) like '%postgres%' 
        )
'''
df = pd.read_sql_query(query_6_3, connection)

In [6]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres.")

В базе содержится 201 вакансия для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [7]:
query_6_4 = f'''
    select
        count(distinct v.id) as cnt
    from
        vacancies as v
    where
        (
            lower(v.name) like '%data scientist%'
            or lower(v.name) like '%data science%'
            or lower(v.name) like '%исследователь данных%'
            or lower(v.name) like '%machine learning%'
            or lower(v.name) like '%машинн%обучен%'
            or (v.name like '%ML%' and v.name not like '%HTML%') 
        ) 
        and lower(v.key_skills) like '%python%'  
'''
df = pd.read_sql_query(query_6_4, connection)

In [8]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для дата-сайентиста, где в качестве ключевого навыка указан Python.")

В базе содержится 351 вакансия для дата-сайентиста, где в качестве ключевого навыка указан Python.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [15]:
query_6_5 = f'''
    select
        round(avg(cardinality), 2) as avg_cnt
    from (
        select
            cardinality(string_to_array(v.key_skills, '\t')) as cardinality
        from
            vacancies as v
        where
            (
                lower(v.name) like '%data scientist%'
                or lower(v.name) like '%data science%'
                or lower(v.name) like '%исследователь данных%'
                or lower(v.name) like '%machine learning%'
                or lower(v.name) like '%машинн%обучен%'
                or (v.name like '%ML%' and v.name not like '%HTML%') 
            ) 
    )   as subquery;  
'''
df = pd.read_sql_query(query_6_5, connection)

In [17]:
print(f"В вакансиях для DS в среднем указывают {df.iloc[0, 0]} ключевых навыка.")

В вакансиях для DS в среднем указывают 6.41 ключевых навыка.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 